In [1]:
import multiprocessing
import os
import time
import numpy as np
from scipy.spatial import cKDTree
import open3d as o3d
import util
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import random
from BendLength import BendLengthCalculator
from preprocess import preProcessData
import pandas as pd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
point_cloud_location = "/home/chris/Code/PointClouds/data/FLIPscans/Bendy/Bendy_1/scan1_Part1.ply"
pcd = o3d.io.read_point_cloud(point_cloud_location)

mesh = o3d.io.read_triangle_mesh("/home/chris/Code/PointClouds/data/FLIPscans/Bendy/BendyCAD.STL")
mesh.compute_vertex_normals()

TriangleMesh with 4156 points and 2104 triangles.

In [ ]:
# Preprocess the point cloud
pcd, average_density, cad_pcd = preProcessData(pcd, mesh, x_rotation=110, z_rotation=90)
pcd_points = np.asarray(pcd.points)

o3d.visualization.draw_geometries([pcd])

[INFO] Average density (excluding sparse points): 288.62 points per 1.0mm sphere
[INFO] Used 998 / 1001 sampled points (≥ 30 neighbors)


In [ ]:
segment_models, segments, segment_indices, main_surface_idx = util.multiOrderRansacAdvanced(pcd, pt_to_plane_dist=0.4, visualize=False, verbose=True)

o3d.visualization.draw_geometries([segments[i] for i in segments])


Identifying main plane
Clustering remaining points
Found 16 disconnected clusters
Filtering clusters
Fitting planes to remaining clusters and filtering


In [16]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.34, ransac_n=3, num_iterations=1000)
main_plane = pcd.select_by_index(inliers)
remaining = pcd.select_by_index(inliers, invert=True)

In [17]:
labels = np.array(remaining.cluster_dbscan(eps=0.8, min_points=50))
num_clusters = labels.max() + 1
print(f"[INFO] Found {num_clusters} disconnected clusters")

# Remove noise (label == -1)
clean_indices = np.where(labels >= 0)[0]
remaining_clean = remaining.select_by_index(clean_indices)
clean_labels = labels[clean_indices]

[INFO] Found 17 disconnected clusters


In [18]:
# Assign new cluster colors
num_clean_clusters = clean_labels.max() + 1

In [19]:
def angle_between_normals(n1, n2):
    cos_angle = np.clip(np.dot(n1, n2), -1.0, 1.0)
    return np.arccos(cos_angle) * 180.0 / np.pi

# Get normal of main plane
main_plane_normal = np.array(plane_model[:3])
main_plane_normal = main_plane_normal / np.linalg.norm(main_plane_normal)

# Collect filtered clusters
filtered_clusters = []
angle_threshold = 15  # degrees

for i in range(num_clean_clusters):
    indices = np.where(clean_labels == i)[0]
    cluster = remaining_clean.select_by_index(indices)

    if len(cluster.points) < 30:
        continue  # Skip tiny/noisy ones

    # Fit plane to cluster
    try:
        cluster_plane, inliers = cluster.segment_plane(distance_threshold=0.4, ransac_n=3, num_iterations=1000)
    except:
        continue

    cluster_normal = np.array(cluster_plane[:3])
    cluster_normal /= np.linalg.norm(cluster_normal)

    angle = angle_between_normals(main_plane_normal, cluster_normal)

    if angle > angle_threshold:
        cluster.paint_uniform_color([0, 1, 0])  # Green for kept
        filtered_clusters.append(cluster)
    else:
        cluster.paint_uniform_color([1, 0, 0])  # Red for removed

# Add back main plane (optional)
main_plane.paint_uniform_color([0.3, 0.3, 1.0])
filtered_clusters.append(main_plane)

# # Visualize
# o3d.visualization.draw_geometries(filtered_clusters)


In [20]:
# # Optional: create a new combined point cloud
# combined = main_plane + sum(filtered_clusters, o3d.geometry.PointCloud())
# o3d.visualization.draw_geometries([combined])


In [ ]:
import matplotlib.pyplot as plt

segment_models = {}
segments = {}
segment_indices = {}
filtered_ids = []
largest_surface_points = 0
main_surface_idx = -1

# Colormap
cmap = plt.get_cmap("tab20")
color_idx = 0

for i in range(num_clean_clusters):
    indices = np.where(clean_labels == i)[0]
    cluster = remaining_clean.select_by_index(indices)

    if len(cluster.points) < 30:
        continue

    try:
        cluster_plane, inliers = cluster.segment_plane(distance_threshold=0.4, ransac_n=3, num_iterations=1000)
    except:
        continue

    cluster_normal = np.array(cluster_plane[:3])
    cluster_normal /= np.linalg.norm(cluster_normal)
    angle = angle_between_normals(main_plane_normal, cluster_normal)

    if angle > angle_threshold:
        color = cmap(color_idx / 20)[:3]
        cluster.paint_uniform_color(color)
        segment_models[i] = cluster_plane
        segments[i] = cluster
        segment_indices[i] = indices
        filtered_ids.append(i)

        if len(cluster.points) > largest_surface_points:
            largest_surface_points = len(cluster.points)
            main_surface_idx = i

        color_idx += 1  # advance for next unique color
    else:
        # Optional: keep red color for discarded ones, or skip saving them
        pass

# Paint and add main surface
main_plane.paint_uniform_color([0.3, 0.3, 1.0])
segments["main_surface"] = main_plane
segment_models["main_surface"] = plane_model
segment_indices["main_surface"] = np.array(inliers)

# Visualize
o3d.visualization.draw_geometries([segments[i] for i in segments])